In [17]:
# Import all relevant libraries to the note book
import osmnx as ox
import pandas as pd
import networkx as nx
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import fiona
import jenkspy

Set Up GeoDataFrame

In [18]:
# Load the walking network with the common code
area_shape = fiona.open("root/Inputs/GemeenteAmsterdam.geojson")
area_shape = shapely.geometry.shape(area_shape[0]['geometry'])

cf = """
     ["area"!~"yes"]
     ["highway"]
     ["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]
     ["foot"!~"no"]
     ["service"!~"private"]
     ["access"!~"private"]
     """

G = ox.graph_from_polygon(polygon=area_shape, custom_filter=cf, simplify=False, retain_all=True, network_type="walk", truncate_by_edge=True)

# Convert to GeoDataFrame
geoDataFrame = ox.graph_to_gdfs(G, nodes=False, edges=True,  node_geometry=False, fill_edge_geometry=True)

In [19]:
#Ensure that the u column is not a index so reset accordingly
geoDataFrame = geoDataFrame.reset_index()
print(geoDataFrame)

                  u            v  key       osmid  oneway       name  \
0           6316199   1360288038    0     4745774   False  Raamsteeg   
1           6316199    451946447    0     7372610   False        NaN   
2           6316199     46388769    0     7372614   False     Singel   
3           6316199   5825714203    0   562720340   False     Singel   
4          16568167   3175727845    0   311836384   False        NaN   
...             ...          ...  ...         ...     ...        ...   
535731  12433165560  12433165518    0   321291099   False    Heining   
535732  12433165560   5030753737    0   321291099   False    Heining   
535733  12433165560  12433165559    0  1343730529   False        NaN   
535734  12433165561   7890327903    0  1343730528   False        NaN   
535735  12433165561  12433165559    0  1343730529   False        NaN   

             highway width  reversed   length  ... bridge tunnel lanes access  \
0           cycleway  1.80      True    5.164  ...    

In [20]:
# To prevent double counting of two way streets the edges are grouped by the same u and v columns and then split again after selecting for the first street
geoDataFrame['edge'] = geoDataFrame.apply(lambda row: tuple(sorted([row['u'], row['v']])), axis=1)
gdf_combined = geoDataFrame.groupby('edge').first().reset_index()
gdf_combined[['u', 'v']] = pd.DataFrame(gdf_combined['edge'].tolist(), index=gdf_combined.index)

print(gdf_combined)

                              edge            u            v  key       osmid  \
0              (6316199, 46388769)      6316199     46388769    0     7372614   
1             (6316199, 451946447)      6316199    451946447    0     7372610   
2            (6316199, 1360288038)      6316199   1360288038    0     4745774   
3            (6316199, 5825714203)      6316199   5825714203    0   562720340   
4           (16568167, 3175727845)     16568167   3175727845    0   311836384   
...                            ...          ...          ...  ...         ...   
267857  (12433165456, 12433165457)  12433165456  12433165457    0    37618224   
267858  (12433165458, 12433165459)  12433165458  12433165459    0    37618224   
267859  (12433165518, 12433165560)  12433165518  12433165560    0   321291099   
267860  (12433165559, 12433165560)  12433165559  12433165560    0  1343730529   
267861  (12433165559, 12433165561)  12433165559  12433165561    0  1343730529   

        oneway       name  

In [8]:
#Store network in case it is needed
with open('./root/Outputs/Network_Amsterdam.geojson' , 'w') as file:
    file.write(gdf_combined.to_json())

In [27]:
#Pull all files from previous script - should be geojson or geo-package
file_mobility = "./root/Outputs/2_modelled_pedestrian_intensity.gpkg" # fill in full path
file_shade_PET = "./root/Outputs/3_Shade+PET.gpkg" # fill in full path

In [28]:
#Read the files into GeoDataFrames and save the columns that are needed only
gdf_mobility = gpd.read_file(file_mobility)
gdf_mobility = gdf_mobility[['u', 'v', 'pop', 'buurtcode', 'young_pop_pct', 'old_pop_pct', 'pop_unknown_pct']]
gdf_mobility['u'] = gdf_mobility['u'].astype('int64')
gdf_mobility['v'] = gdf_mobility['v'].astype('int64')
gdf_shade = gpd.read_file(file_shade_PET)
gdf_shade = gdf_shade[['u', 'v', "0900",  "0930", "1000", "1030", "1100", "1130", "1200", "1230", "1300", "1330", "1400", "1430", "1500", "1530", "1600", "1630", "1700", "1730", "1800", "1830", "1900", "1930", "2000", "sum_adjust", 'PET', "avg_exposure_percent", 'avg_shade_percent']]

In [29]:
#Again to prevent double counting the first street is selected
gdf_shade['edge'] = gdf_shade.apply(lambda row: tuple(sorted([row['u'], row['v']])), axis=1)
gdf_shade_combined = gdf_shade.groupby('edge').first().reset_index()
gdf_shade_combined [['u', 'v']] = pd.DataFrame(gdf_combined['edge'].tolist(), index=gdf_combined.index)

In [ ]:
print(gdf_shade_combined)
print(gdf_mobility)
print(gdf_combined)

                  u           v      0900      0930      1000      1030  \
0           6316199  1360288038  0.073318  0.010515  0.062450  0.107095   
1           6316199   451946447  0.144737  0.074230  0.189317  0.160753   
2           6316199    46388769  0.017194  0.007147  0.016331  0.023917   
3           6316199  5825714203  0.053952  0.005449  0.046026  0.080215   
4          16568167  3175727845  0.960552  0.960552  0.977149  0.985047   
...             ...         ...       ...       ...       ...       ...   
535287  12397581807  1426936884  0.639661  0.567187  0.676846  0.692416   
535288  12398264801   350431665  0.132534  0.039515  0.131939  0.118753   
535289  12398264801  4653799513  0.198753  0.087807  0.208488  0.274880   
535290  12398264802   311590674  0.099197  0.009920  0.100000  0.100000   
535291  12398264802   350431665  0.100000  0.010000  0.100000  0.100000   

            1100      1130      1200      1230  ...      1730      1800  \
0       0.096896  0.0742

In [30]:
#Combine all GeoDataFrames into one file
gdf_heat_combined = gdf_combined.merge(gdf_shade_combined, how = 'left', on =['u', 'v']) 
gdf_heat_combined = gdf_heat_combined.merge(gdf_mobility, how = 'left', on =['u', 'v']) 
print(gdf_heat_combined.head(30))

                     edge_x         u            v  key      osmid  oneway  \
0       (6316199, 46388769)   6316199     46388769    0    7372614   False   
1      (6316199, 451946447)   6316199    451946447    0    7372610   False   
2     (6316199, 1360288038)   6316199   1360288038    0    4745774   False   
3     (6316199, 5825714203)   6316199   5825714203    0  562720340   False   
4    (16568167, 3175727845)  16568167   3175727845    0  311836384   False   
5   (16568167, 10633464529)  16568167  10633464529    0  311836398   False   
6      (25596455, 46356773)  25596455     46356773    0   37481780   False   
7    (25596455, 8383889398)  25596455   8383889398    0   37481780   False   
8    (25596455, 8383889399)  25596455   8383889399    0   37481778   False   
9    (25596455, 8383889401)  25596455   8383889401    0   37481778   False   
10   (25596477, 2739513281)  25596477   2739513281    0   23289375   False   
11   (25596477, 2739513283)  25596477   2739513283    0  1139529

Set Geometry

In [31]:
#Set the geometry as it is defined by the first network pulled for the GeoDataFrame
gdf_heat_combined = gdf_heat_combined.set_geometry(gdf_heat_combined["geometry"], inplace=False)
gdf_heat_combined = gdf_heat_combined.fillna(0)

Generate threshold functions

In [32]:
#Fill in strings for column names
#Funtion which loops through various scores and sets the column value to 1 or 0 accordingly for threshold functionality
def shade_treshold(value):
    if value < 0.4:
        return 1
    else:
        return 0

def PET_treshold(value):
    if value >= 35:
        return 1
    else:
        return 0
    
def Business_bound(value):
    if value != 0:
        return 1
    else:
        return 0

def top_percent(value):
    if value > 0.96:
        return 1
    else:
        return 0

Do the math

In [33]:
#Function calculates our score based on the columns from the GeoDataFrame
def calculate_score(gdf_combined, column1_shade_sum, column2_PET, column3_mob, column1_shade_perc):
    max_shade = gdf_combined[column1_shade_sum].max()
    min_shade = gdf_combined[column1_shade_sum].min()
    max_PET = gdf_combined[column2_PET].max()
    min_PET = gdf_combined[column2_PET].min()


    #Set column to 0 or 1 based on threshold values above
    gdf_combined['shade_tresh'] = gdf_combined[column1_shade_perc].apply(shade_treshold)
    gdf_combined['PET_tresh'] = gdf_combined[column2_PET].apply(PET_treshold)

    #Reprojected Shade and PET score from 0-1 to make them comparable
    gdf_combined['shade_score'] = ((gdf_combined[column1_shade_sum]-min_shade)/(max_shade-min_shade)) 
    gdf_combined['PET_score'] = ((gdf_combined[column2_PET]-min_PET)/(max_PET-min_PET))

    #Calculate final risk score per street
    gdf_combined['Final_score_all'] = gdf_combined['shade_tresh']*gdf_combined['PET_tresh']*(((gdf_combined['shade_score']) + (gdf_combined['PET_score']))/2) #Add weights to criteria if wanted
    
    #Calulate top 4% percent of streets in Amsterdam based on modelled intensity, requirement that Final_score_all is not 0
    gdf_combined['busy_select'] = (gdf_combined['Final_score_all'].apply(Business_bound))
    gdf_combined['busy_after_crit'] = gdf_combined[column3_mob]*gdf_combined['busy_select']
    gdf_combined['busy_after_crit'] = gdf_combined['busy_after_crit'].rank(pct = True)
    gdf_combined['busy_treshold'] =  gdf_combined['busy_after_crit'].apply(top_percent)
    gdf_combined['Final_score_4_perc'] = gdf_combined['Final_score_all']*gdf_combined['busy_treshold']

    #Add column defining with category when displaying the flow values later
    gdf_jenks = gdf_combined.loc[gdf_combined[column3_mob] != 0]
    jnb = jenkspy.JenksNaturalBreaks(3)
    jnb.fit(gdf_jenks[column3_mob])
    brea = jnb.breaks_
    breaki = [0] + brea
    gdf_combined['jenkins_bin'] = pd.cut(gdf_combined[column3_mob], bins=breaki, labels=['bin_1', 'bin_2', 'bin_3', 'bin_4'], include_lowest = True) 
    
    return gdf_combined

In [34]:
#Implementing the previous function
gdf_heat_combined = calculate_score(gdf_heat_combined, 'sum_adjust', 'PET', 'pop', 'avg_shade_percent')
gdf_heat_combined = gdf_heat_combined.round(decimals=3)

print(gdf_heat_combined.head(30))

                     edge_x         u            v  key      osmid  oneway  \
0       (6316199, 46388769)   6316199     46388769    0    7372614   False   
1      (6316199, 451946447)   6316199    451946447    0    7372610   False   
2     (6316199, 1360288038)   6316199   1360288038    0    4745774   False   
3     (6316199, 5825714203)   6316199   5825714203    0  562720340   False   
4    (16568167, 3175727845)  16568167   3175727845    0  311836384   False   
5   (16568167, 10633464529)  16568167  10633464529    0  311836398   False   
6      (25596455, 46356773)  25596455     46356773    0   37481780   False   
7    (25596455, 8383889398)  25596455   8383889398    0   37481780   False   
8    (25596455, 8383889399)  25596455   8383889399    0   37481778   False   
9    (25596455, 8383889401)  25596455   8383889401    0   37481778   False   
10   (25596477, 2739513281)  25596477   2739513281    0   23289375   False   
11   (25596477, 2739513283)  25596477   2739513283    0  1139529

Export as GEOJson

In [35]:
#Export file as GEOJson
with open('./root/Outputs/4_Final_Heat_Risk_Score.geojson' , 'w') as file:
    file.write(gdf_heat_combined.to_json())